In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from libraries.import_export_data_objects import import_export_data as Import_Export_Data
from libraries.altair_renderings import AltairRenderings
from libraries.utility import Utility
import os
import altair as alt
my_altair = AltairRenderings()

In [3]:
my_json,my_map = my_altair.get_world_map()
my_map

alt.Chart(...)

In [4]:
!pip install geopandas >/dev/null
!pip install gpdvega >/dev/null
!pip install configparser >/dev/null
import altair as alt
import geopandas as gpd
import gpdvega
import numpy as np
import matplotlib.pyplot as plt
import configparser
import pandas as pd

In [5]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

interested_countries = ['Australia','Brazil','Canada','China','France','Germany','India','Indonesia','Iran','Italy','Japan',
                       'Mexico','Netherlands','Russia','Saudi Arabia','South Korea','Spain','Switzerland','United Kingdom','United States of America']

gdp = list(range(100,300,10))

country_gdp = pd.DataFrame(
    {'name': interested_countries,
     'GDP': gdp
    })

world_gdp = pd.merge(world, country_gdp, on='name', how = 'outer')
world_gdp['GDP'] = world_gdp['GDP'].fillna(0)

world_gdp.head()


,pop_est,continent,name,iso_a3,gdp_md_est,geometry,GDP
0,889953.0,Oceania,Fiji,FJI,5496,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",0.0
1,58005463.0,Africa,Tanzania,TZA,63177,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",0.0
2,603253.0,Africa,W. Sahara,ESH,907,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",0.0
3,37589262.0,North America,Canada,CAN,1736425,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",120.0
4,328239523.0,North America,United States of America,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",290.0


In [6]:
from libraries.import_export_data_objects import import_export_data as Import_Export_Data
from libraries.altair_renderings import AltairRenderings
from libraries.utility import Utility
import os
import altair as alt
import pandas as pd
my_altair = AltairRenderings()
my_data = Import_Export_Data()

In [7]:
all_gdp=my_data.get_gdp_all_data()

In [8]:
year2020 = all_gdp[all_gdp['Year'] == 2020]
top20_2020 = year2020.sort_values(['GDP'], ascending=False).head(20)[['Country','GDP']]

In [9]:
len(top20_2020)

20

In [10]:
from vega_datasets import data

world_map_source = alt.topo_feature(data.world_110m.url, 'countries')

In [11]:
country_source = my_data.get_world_countries_by_iso_label()
country_source.loc[84,'Country'] = 'South Korea'
country_source = country_source.drop(4)
country_gdp = pd.merge(country_source, top20_2020, on='Country', how = 'outer')

In [12]:
country_gdp['GDP'] = country_gdp['GDP'].fillna(0)

In [13]:
chart = alt.Chart(world_map_source).mark_geoshape(
    stroke='black',
    strokeWidth=1.10
).project('mercator'
).encode(
    color = alt.condition(
        'datum.GDP > 0',
        alt.Color('GDP:Q',legend=alt.Legend(title="County GDP in $MM")),
        alt.value('lightgrey')
    ),
    tooltip=[alt.Tooltip("Country:N", title="Country")]
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(country_gdp, "id", ["Country",'GDP']),
).properties(
    width=700,
    height=500
)
    
chart

alt.Chart(...)